In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam

# Load Fashion MNIST dataset
(X_train, _), (_, _) = fashion_mnist.load_data()

# Normalize dataset
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)

# Set up GAN parameters
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

# Build generator model
def build_generator():
    generator = Sequential([
        Dense(128 * 7 * 7, input_dim=latent_dim),
        LeakyReLU(alpha=0.01),
        Reshape((7, 7, 128)),
        Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'),
        LeakyReLU(alpha=0.01),
        Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'),
        LeakyReLU(alpha=0.01),
        Conv2DTranspose(channels, kernel_size=3, strides=2, padding='same', activation='tanh')
    ])
    return generator

# Build discriminator model
def build_discriminator():
    discriminator = Sequential([
        Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding='same'),
        LeakyReLU(alpha=0.01),
        Dropout(0.4),
        Conv2D(128, kernel_size=3, strides=2, padding='same'),
        LeakyReLU(alpha=0.01),
        Dropout(0.4),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return discriminator

# Compile both networks in the GAN
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=Adam(learning_rate=0.0001, beta_1=0.5),
                          metrics=['accuracy'])

    # Combined model (stacked generator and discriminator)
    discriminator.trainable = False
    gan = Sequential([
        generator,
        discriminator
    ])

    gan.compile(loss='binary_crossentropy',
                optimizer=Adam(learning_rate=0.0001, beta_1=0.5))

    return gan

# Plot generated images
def plot_generated_images(generator, epoch, examples=25, dim=(5, 5), figsize=(10, 10)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = generated_images * 0.5 + 0.5  # Rescale images to 0-1 range

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()

# Train the GAN
def train_gan(epochs, batch_size=128, save_interval=1):
    # Build and compile the discriminator
    discriminator = build_discriminator()

    # Build and compile the generator
    generator = build_generator()

    # Build and compile GAN model
    gan = build_gan(generator, discriminator)

    # Training loop
    batch_count = X_train.shape[0] // batch_size
    d_losses = []
    g_losses = []
    for epoch in range(1, epochs+1):
        print(f"Epoch {epoch}/{epochs}")

        for _ in range(batch_count):
            # Generate random noise as input for the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))

            # Generate fake images from the noise
            generated_images = generator.predict(noise)

            # Get a random set of real images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_images = X_train[idx]

            # Concatenate real and fake images to form the batch for discriminator
            X = np.concatenate([real_images, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9  # One-sided label smoothing

            # Train discriminator
            discriminator.trainable = True
            d_loss, d_acc = discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        # Print discriminator loss, accuracy, and generator loss
        print("[Discriminator loss: {:.6f}, accuracy.: {:.2f}%] [Generator loss: {:.6f}]".format(d_loss, 100 * d_acc, g_loss))
        d_losses.append(d_loss)
        g_losses.append(g_loss)

        # Plot generated images at save_interval
        if epoch % save_interval == 0:
            plot_generated_images(generator, epoch)

    # Plot the losses
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label='Discriminator loss')
    plt.plot(g_losses, label='Generator loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('GAN Loss')
    plt.legend()
    plt.savefig('gan_loss_plot.png')
    plt.show()

# Run the function to train the GAN
def run_gan():
    epochs = 50
    batch_size = 128
    save_interval = 1
    train_gan(epochs=epochs, batch_size=batch_size, save_interval=save_interval)

# Run the GAN
run_gan()
